In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
import time 
import re
import pandas as pd

In [2]:
code = pd.read_excel('./code.xlsx', header = 0, dtype=str)
code_list = code['code'].tolist()
code_list

['000690',
 '000020',
 '000019',
 '000017',
 '000016',
 '000014',
 '000012',
 '000011',
 '000150',
 '000009',
 '000008',
 '000007',
 '000006',
 '000005',
 '000004',
 '000002',
 '000042',
 '000528',
 '000021',
 '000040',
 '000039',
 '000038',
 '000037',
 '000036',
 '000035',
 '000059',
 '000032',
 '000031',
 '000030',
 '000029',
 '000028',
 '000027',
 '000026',
 '000025',
 '002014',
 '000069',
 '000068',
 '000066',
 '000065',
 '000063',
 '000062',
 '000061',
 '000060',
 '000034',
 '000058',
 '000056',
 '000055',
 '000050',
 '000049',
 '000048',
 '000419',
 '000166',
 '000046',
 '000159',
 '000158',
 '000157',
 '000156',
 '000155',
 '000153',
 '000151',
 '000010',
 '000100',
 '000099',
 '000096',
 '000090',
 '000089',
 '000088',
 '000078',
 '002502',
 '000417',
 '000416',
 '000415',
 '000413',
 '000411',
 '000410',
 '000409',
 '000538',
 '000407',
 '000404',
 '000403',
 '000402',
 '000401',
 '000400',
 '000338',
 '000547',
 '000333',
 '000070',
 '000504',
 '000503',
 '000502',
 '000501',

In [9]:
#for code in code_list:

driver = webdriver.Chrome('/Users/jiacheng/Downloads/chromedriver')
# a new window for QA site
driver2 = webdriver.Chrome('/Users/jiacheng/Downloads/chromedriver')
url = 'http://irm.cninfo.com.cn/ircs/index'



for code in code_list:

    print('Collecting data for company ... ' + code + '...........')

    driver.set_window_size(1000,30000)

    driver.get(url)
    table_content_translate_box = []
    #while(True): # Stop manually 

    # scoll down
    #driver.execute_script("window.scrollBy(0,8000)")
    #time.sleep(1)

    # find search box and search company code
    time.sleep(2)
    searc_box = driver.find_element_by_class_name('el-input__inner')
    searc_box.click()
    time.sleep(1)
    searc_box.send_keys(code)
    searc_box.send_keys(Keys.ENTER)
    time.sleep(1)

    # go to QA section
    tablist = driver.find_element_by_css_selector('.el-tabs__nav.is-top')
    tablist.find_element_by_id('tab-2').click()

    # find date search
    date_search_box = driver.find_element_by_class_name('el-range-input')
    date_search_box.click()
    date_search_box.clear()
    time.sleep(1)
    date_search_box.send_keys('2012-01-01')
    time.sleep(1)
    action = ActionChains(driver)
    action.move_by_offset(0,0).click().perform()


    # max page number
    page = driver.find_element_by_class_name('el-pager')
    pages = page.find_elements_by_class_name('number')
    page_number = []
    for item in pages:
        page_number.append(int(item.text))
        #print(item.text)

    max_page_number = max(page_number)
    #print(max_page_number)

    #data_ids = []

    # header
    result = []
    header = ['深市代码', '提问', '提问时间', '回复', '回复时间']
    result.append(header)

    while True:
        # current page number
        page = driver.find_element_by_class_name('el-pager')
        active_page_number = page.find_element_by_css_selector('.number.active').text
        print("Current page:" + str(max_page_number) + '/' + str(active_page_number))


        # find QA of this page 
        search_content_box = driver.find_elements_by_class_name('search-content-box')
        time.sleep(1)
        for item in search_content_box:
            temp_item1 = item.find_element_by_css_selector('.item-question.left-content.table-list.pd-r-20')
            temp_item2 = temp_item1.find_element_by_css_selector('.table-content.div-translate')
            data_id = temp_item2.get_attribute('data-id')
            print(data_id)
            #data_ids.append(data_id)
            time.sleep(1)

            # get QA for each data id
            url_QA = 'http://irm.cninfo.com.cn/ircs/question/questionDetail?questionId=' + data_id
            driver2.get(url_QA)
            time.sleep(3)
            # have Q and A
            Q = driver2.find_element_by_css_selector('.question-detail.translate-box.pd-30').text
            time.sleep(1)
            print(Q)
            Qtime = driver2.find_element_by_class_name('question-time').text.split('·')[0]
            time.sleep(1)
            print(Qtime)
            try:
                A = driver2.find_element_by_css_selector('.left-content.no-data').text
                print(A)
                Atime = '暂无回复'
                print(Atime)
            except Exception:

                A = driver2.find_element_by_css_selector('.reply-content.translate-box').text
                print(A)

                Atime = driver2.find_element_by_css_selector('.question-time.hidden-xs-only').text.split('：')[1]
                print(Atime)
            print('________________')
            time.sleep(2)
            result.append([code, Q, Qtime, A, Atime])


        print('Page finished')

        # next page
        next_btn = driver.find_element_by_class_name('btn-next')
        next_btn.find_element_by_css_selector('.el-icon.el-icon-arrow-right').click()
        time.sleep(1)
        if int(active_page_number) == max_page_number:
            print("All pages done")
            break
    # export to csv        
        df = pd.DataFrame(result)
        df.to_csv('./'+code+'.csv', index = False, header = 0)
        


#driver2 = webdriver.Chrome('/Users/jiacheng/Downloads/chromedriver')
#driver2.get('http://irm.cninfo.com.cn/ircs/question/questionDetail?questionId=1210123083781349376')


#driver.get('http://irm.cninfo.com.cn/ircs/question/questionDetail?questionId=1210123083781349376')
# Click
#show_more_box = driver.find_elements_by_class_name('show_more_box')
#driver.find_element_by_class_name('cursor-pointer').click()

# find Q and A
#table_content = driver.find_elements_by_css_selector(".left-content.table-list")
#table_content_translate_box.extend(driver.find_elements_by_css_selector(".table-content.translate-box"))
#content = driver.find_elements_by_class_name('table-content translate-box')

    
#driver2 = webdriver.Chrome('/Users/jiacheng/Downloads/chromedriver')
#driver2.get('http://irm.cninfo.com.cn/ircs/question/questionDetail?questionId=1210123083781349376')

Current page:101/1
1239248647509561344
银保监会发布了关于银行业数字化转型指导意见，近期也发生了一些地区村镇银行的风控问题，请问客商银行对于数字化风控建设有何进展？
2022-07-28
暂无回复
暂无回复
________________
1230093091495038976
尊敬的邹董事长: 据了解，目前虚拟电厂已在国内多个地方开展示范应用。2022年5月，国电投深圳能源发展有限公司的虚拟电厂平台参与电力现货市场，获得平均度电0.274元的收益，成为我国首个调度用户负荷参与电力现货市场并盈利的虚拟电厂。 请问公司是否有进军虚拟电厂的考量？
2022-07-16
您好，感谢您的关注与支持！ 公司暂未投资虚拟电厂。未来公司将在做大做强能源电力核心主业的基础上，关注和探索发展新型智慧能源、储能节能、新型能源服务技术等新型战略产业，谢谢！
2022-07-21
________________
1230829841593126912
请问贵公司在节能减排方面未来有何规划？
2022-07-17
您好，感谢您的关注与支持！ 公司始终坚持走节能减排的发展之路，发展高效清洁能源和可再生能源，积极推进科技创新，提高资源利用效率，成功打造国内新能源电力的典范，实现了经济效益与环境效益的完美结合。 公司下属全资子公司广东宝丽华电力有限公司、陆丰宝丽华新能源电力有限公司各项排放指标均优于国家环保排放标准要求，谢谢！
2022-07-21
________________
1203392233297350656


KeyboardInterrupt: 

In [72]:
Atime.split('：')[1]

['回复时间', '2022-05-27']

In [37]:
# 

driver2 = webdriver.Chrome('/Users/jiacheng/Downloads/chromedriver')

for data_id in data_ids:
    url_QA = 'http://irm.cninfo.com.cn/ircs/question/questionDetail?questionId=' + data_id

    driver2.get(url_QA)
    time.sleep(2)
    # have Q and A
    Q = driver2.find_element_by_css_selector('.question-detail.translate-box.pd-30').text
    print(Q)
    Qtime = driver2.find_element_by_class_name('question-time').text.split('·')[0]
    print(Qtime)
    try:
        A = driver2.find_element_by_css_selector('.left-content.no-data').text
        print(A)
        Atime = '暂无回复'
        print(Atime)
    except Exception:

        A = driver2.find_element_by_css_selector('.reply-content.translate-box').text
        print(A)

        Atime = driver2.find_element_by_css_selector('.question-time.hidden-xs-only').text.split('：')[1]
        print(Atime)
    print('________________')
    time.sleep(2)
    result.append([code, Q, Qtime, A, Atime])



NameError: name 'data_ids' is not defined

In [43]:
driver2 = webdriver.Chrome('/Users/jiacheng/Downloads/chromedriver')
driver2.get(url_QA)
time.sleep(2)
A = driver2.find_element_by_css_selector('.left-content.no-data').text
print(A)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".left-content.no-data"}
  (Session info: chrome=103.0.5060.114)


In [32]:
temp_item1 = search_content_box[0].find_element_by_css_selector('.item-question.left-content.table-list.pd-r-20')
temp_item2 = temp_item1.find_element_by_css_selector('.table-content.div-translate')
data_id = temp_item2.get_attribute('data-id')
data_id

'1221178010696982528'

In [31]:
temp_item2.get_attribute('data-id')

'1221178010696982528'

In [33]:
driver2 = webdriver.Chrome('/Users/jiacheng/Downloads/chromedriver')
driver2.get('http://irm.cninfo.com.cn/ircs/question/questionDetail?questionId=1221178010696982528')

In [86]:
text = table_content_translate_box[0].get_attribute('data-id')
text

'1213978942209929216'

In [98]:
driver.get('http://irm.cninfo.com.cn/ircs/question/questionDetail?questionId=1210123083781349376')